In [2]:
import dotenv
import os
dotenv.load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
# os.getenv('SEARCH_ENGINE_ID')

In [3]:
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import BeautifulSoupTransformer

async def load_html():
    loader = AsyncHtmlLoader(["https://en.wikipedia.org/wiki/OpenAI"])
    html = loader.load()
    bs_transformer = BeautifulSoupTransformer()
    docs_transformed = bs_transformer.transform_documents(html, tags_to_extract=["span"])
    return docs_transformed
docs_transformed = await load_html()
doc_string = ""
for doc in docs_transformed:
    doc_string += doc.page_content


USER_AGENT environment variable not set, consider setting it to identify your requests.
Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.26it/s]


In [4]:
doc_string

'Main menu Main page Contents Current events Random article About Wikipedia Contact us Help Learn to edit Community portal Recent changes Upload file    Search Donate Appearance Create account Log in Personal tools Create account Log in learn more Contributions Talk 1 History Toggle History subsection 1.1 2015–2018: Non-profit beginnings 1.2 2019: Transition from non-profit 1.3 2020–2023: ChatGPT, DALL-E, partnership with Microsoft 1.4 2024–present: Public/non-profit efforts, Sora, partnership with Apple 2 Participants Toggle Participants subsection 2.1 Key employees 2.2 Board of directors of the OpenAI nonprofit 2.3 Principal individual investors [ 104 ] 2.4 Corporate investors 3 Motives 4 Strategy 5 Products and applications Toggle Products and applications subsection 5.1 Reinforcement learning 5.1.1 Gym 5.1.1.1 Gym Retro 5.1.2 RoboSumo 5.1.3 OpenAI Five 5.1.4 Dactyl 5.2 API 5.3 Text generation 5.3.1 OpenAI\'s original GPT model ("GPT-1") 5.3.2 GPT-2 5.3.3 GPT-3 5.3.4 Codex 5.3.5 GPT

In [5]:
import spacy
def overlapping_chunking(doc_string, max_sentences, overlapping_sentences):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(doc_string)
    sentences = [s.text for s in doc.sents]
    chunks = []
    curr_ind = 0
    while curr_ind < len(sentences):
        endpoint = min(curr_ind + max_sentences, len(sentences))
        chunk = ' '.join(sentences[curr_ind: endpoint])
        chunks.append(chunk)
        curr_ind += max_sentences - overlapping_sentences
    return chunks
chunks = overlapping_chunking(doc_string, 8, 2)
len(chunks)

284

In [6]:
max_chunk_size = float('-inf')
for chunk in chunks:
    max_chunk_size = max(max_chunk_size, len(chunk))
max_chunk_size

2513

In [7]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
print(os.environ["PYTORCH_ENABLE_MPS_FALLBACK"])

1


In [8]:
import torch
from transformers import pipeline
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# device="cpu"
# summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device="cpu")
# max_chunk_size = 1000
# inputs = summarizer.tokenizer(doc_string, return_tensors="pt", truncation=False)
# tokens = inputs.input_ids[0]
# chunks = [tokens[i:i+max_chunk_size] for i in range(0, len(tokens), max_chunk_size)]
# summaries = []
# for chunk in chunks:
#     chunk_text = summarizer.tokenizer.decode(chunk, skip_special_tokens=True)
#     summary = summarizer(chunk_text, truncation=True, max_length=150)[0]['summary_text']
#     summaries.append(summary)
# final_summary = " ".join(summaries)
# final_summary


KeyboardInterrupt: 

In [9]:
import openai
def prompt_llm_for_related_topics(final_summary):
    prompt = f"""
    Based on the following content: {final_summary} generate exactly 3 topics
    that are related to this topic or discussed in the article summary. 
    Provide your answer in the format '['Topic 1', 'Topic 2', 'Topic 3']'
    """


    response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant who generates topics related to article summaries."},
        {"role": "user", "content": prompt},
    ]
    )
    answer = response.choices[0].message.content
    return answer
answer = prompt_llm_for_related_topics(final_summary)
answer

"['OpenAI's Development of ChatGPT', 'Microsoft's Investments in OpenAI', 'Challenges and Controversies Surrounding OpenAI']"

In [87]:
print(' '.join(final_summary.split()[800:900]))

hype" (https://www.theverge.com/2017/8/14/16141938/dota-2-openai-bots-elon-musk-artificial-intelligence) from the original on June 26, 2018. "#cite_ref-129) Savov, Vlad (August 14, 2017). "My favoritegame has been invasion by killerAI bots and Musk hype." (https:www. theverger.co.uk/news/articles/2018-06-25/musK-backed-bot-conquers-e-gamer-teams-in-ai-break "The International 2018: Results" (https://blog.openai.com/the-international-2018-results/). blog.openAI.com. July 18, 2018. Archived on February 13, 2019. (#cite_ref-137) Vincent, James (June 25, 2018). "AI bots trained for 180 years a day to beat humans at Dota 2" "Pro Gamers Fend off Elon Musk-Backed AI Bots—for Now" OpenAI's Dota 2 bot defeated 99.4% of players in public matches, according to Venture Beat. #cite_ref-145: Fangasadha, Edbert Felix, Soeroredjo, Steffi; Anderies; Gunawan, Alexander Agung Santoso (September 17, 2022).


In [53]:
chunks[0]

'Main menu Main page Contents Current events Random article About Wikipedia Contact us Donate Help Learn to edit Community portal Recent changes Upload file     Search Appearance Create account Log in Personal tools Create account Log in learn more Contributions Talk 1 History Toggle History subsection 1.1 2015–2018: Non-profit beginnings 1.2 2019: Transition from non-profit 1.3 2020–2023: ChatGPT, DALL-E, partnership with Microsoft 1.4 2024–present: Public/non-profit efforts, Sora, partnership with Apple 2 Participants Toggle Participants subsection 2.1 Key employees 2.2 Board of directors of the OpenAI nonprofit 2.3 Principal individual investors [ 99 ] 2.4 Corporate investors 3 Motives 4 Strategy 5 Products and applications Toggle Products and applications subsection 5.1 Reinforcement learning 5.1.1 Gym 5.1.1.1 Gym Retro 5.1.2 RoboSumo 5.1.3 OpenAI Five 5.1.4 Dactyl 5.2 API 5.3 Text generation 5.3.1 OpenAI\'s original GPT model ("GPT-1") 5.3.2 GPT-2 5.3.3 GPT-3 5.3.4 Codex 5.3.5 GPT

In [105]:
import openai
prompt = f"""
Based on the following content: {final_summary} generate 10-15 questions 
that will help readers understand the content better then provide
informative answers to these questions.
Provide these answers in the format Question: Answer
"""


response = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
      {"role": "system", "content": "You are a helpful assistant who generates FAQs from website content."},
      {"role": "user", "content": prompt},

  ]
)
# print(response.choices[0].text.strip())
# response['choices'][0]['message']['content']

In [106]:
import re
answer = response.choices[0].message.content
faq_list = re.split(r"\n+", answer)
faq_list

['Question: What are some of the key models developed by OpenAI?',
 'Answer: OpenAI has developed models such as GPT-1, GPT-2, GPT-3, Codex, Whisper, Music generation models like MuseNet and Jukebox, as well as ChatGPT and Stargate supercomputers.',
 "Question: What is OpenAI's revenue situation and recent funding developments?",
 "Answer: OpenAI's losses doubled to around $540 million in the previous year, and the company has raised $124 million in funding so far. It is expected to reach $1 billion in revenue by 2024 and is in talks to secure an additional $100 million in new funding.",
 'Question: What are some notable collaborations and partnerships involving OpenAI?',
 'Answer: OpenAI has partnerships with Microsoft, including significant investments in the company. It has also announced partnerships with universities and has collaborated with Reddit, Vox, and The Atlantic. ',
 'Question: What are some controversies surrounding OpenAI?',
 'Answer: OpenAI has faced controversies suc

In [3]:
faq_list = ['Question: What are some of the key models developed by OpenAI?',
 'Answer: OpenAI has developed models such as GPT-1, GPT-2, GPT-3, Codex, Whisper, Music generation models like MuseNet and Jukebox, as well as ChatGPT and Stargate supercomputers.',
 "Question: What is OpenAI's revenue situation and recent funding developments?",
 "Answer: OpenAI's losses doubled to around $540 million in the previous year, and the company has raised $124 million in funding so far. It is expected to reach $1 billion in revenue by 2024 and is in talks to secure an additional $100 million in new funding.",
 'Question: What are some notable collaborations and partnerships involving OpenAI?',
 'Answer: OpenAI has partnerships with Microsoft, including significant investments in the company. It has also announced partnerships with universities and has collaborated with Reddit, Vox, and The Atlantic. ',
 'Question: What are some controversies surrounding OpenAI?',
 'Answer: OpenAI has faced controversies such as legal action from Elon Musk, concerns about technological transparency, copyright infringement accusations in training data, and issues with content moderation contracts. The company has also been criticized for its involvement in state-backed influence operations.',
 "Question: What is the significance of AGI and Q* in OpenAI's research?",
 'Answer: OpenAI is focused on achieving Artificial General Intelligence (AGI), which refers to autonomous systems surpassing humans in economically valuable tasks. Q* is seen as a potential breakthrough in AGI research, with some believing it could be an AI breakthrough ahead of leadership changes at the company.']
questions = [faq for i, faq in enumerate(faq_list) if i % 2 == 0]
answers = [faq for i, faq in enumerate(faq_list) if i % 2 == 1]

['Answer: OpenAI has developed models such as GPT-1, GPT-2, GPT-3, Codex, Whisper, Music generation models like MuseNet and Jukebox, as well as ChatGPT and Stargate supercomputers.',
 "Answer: OpenAI's losses doubled to around $540 million in the previous year, and the company has raised $124 million in funding so far. It is expected to reach $1 billion in revenue by 2024 and is in talks to secure an additional $100 million in new funding.",
 'Answer: OpenAI has partnerships with Microsoft, including significant investments in the company. It has also announced partnerships with universities and has collaborated with Reddit, Vox, and The Atlantic. ',
 'Answer: OpenAI has faced controversies such as legal action from Elon Musk, concerns about technological transparency, copyright infringement accusations in training data, and issues with content moderation contracts. The company has also been criticized for its involvement in state-backed influence operations.',
 'Answer: OpenAI is focu

In [104]:
len(faq_list)

28

In [11]:
import requests
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
SEARCH_ENGINE_ID = "f35b11676565b4fb5"
def search_google(query):
    url = f"https://www.googleapis.com/customsearch/v1"
    params = {
        "key": GOOGLE_API_KEY,
        "cx": SEARCH_ENGINE_ID,
        "q": query,
    }
    response = requests.get(url, params=params)
    results = response.json()
    return results
cmc_query = search_google("Christian Mccaffery injury status")

In [13]:
cmc_query

{'kind': 'customsearch#search',
 'url': {'type': 'application/json',
  'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'},
 'queries': {'request': [{'title': 'Google Custom Search - Christian Mccaffery injury status',
    'totalResults': '91400',
    'searchTerms': 'Christian Mccaffery injury status',
    'count': 10,
    'startIndex': 1,
    'inputEncoding': 'utf8',
    'outp

In [18]:
for i in range(10):
    print(cmc_query['items'][i]['title'])
    print(cmc_query['items'][i]['link'])

Christian McCaffrey - NFL News, Rumors, & Updates | FOX Sports
https://www.foxsports.com/nfl/christian-mccaffrey-player
Christian McCaffrey injury update: Will 49ers RB be ready for Week ...
https://www.sportingnews.com/us/nfl/news/christian-mccaffrey-injury-49ers-rb-week-1-regular-season/3e1add7aca7976c53674df9b
Christian McCaffrey Injury Update: 'If We had a Game, I'd play ...
https://www.siriusxm.com/blog/christian-mccaffrey-calf-strain-nfl-radio
Christian McCaffrey Injury Update: Latest on 49ers RB' Status For ...
https://www.profootballnetwork.com/christian-mccaffrey-injury-update-49ers-rb-week-1-monday-fantasy-2024/
Christian McCaffrey Injury History & Updates
https://www.draftsharks.com/fantasy/injury-history/christian-mccaffrey/7750
Christian McCaffrey gives an update on his calf injury - NBC Sports
https://www.nbcsports.com/nfl/profootballtalk/rumor-mill/news/christian-mccaffrey-gives-an-update-on-his-calf-injury
49ers' Christian McCaffrey Gives Calf Injury Update, Says He'd P

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
# os.getenv("GOOGLE_API_KEY")

True

In [3]:
import tensorflow as tf
if tf.config.list_physical_devices('MPS'):
    print("MPS device found")

In [9]:
import requests
response = requests.head("https://arxiv.org/pdf/2307.06435", allow_redirects=True)
content_type = response.headers.get('Content-Type', '').lower()
content_type

'application/pdf'

In [10]:
from langchain.document_loaders import PyPDFLoader
from io import BytesIO
import os
from tempfile import NamedTemporaryFile
response = requests.get("https://arxiv.org/pdf/2307.06435")
response.raise_for_status()
# pdf_content = BytesIO(response.content)
with NamedTemporaryFile(suffix=".pdf", delete=False) as temp_pdf:
    temp_pdf.write(response.content)
    temp_pdf_path = temp_pdf.name
loader = PyPDFLoader(temp_pdf_path)
documents = loader.load()
os.remove(temp_pdf_path)
# return documents
doc_string = ""
for doc in documents:
    doc_string += doc.page_content

In [11]:
from PyPDF2 import PdfReader
pdf_content = BytesIO(response.content)
reader = PdfReader(pdf_content)
metadata = reader.metadata

title = metadata.title if metadata.title else ""
title

'A Comprehensive Overview of Large Language Models'

In [12]:
len(doc_string)

268055

In [13]:
def load_pdf(url):
    response = requests.get(url)
    with NamedTemporaryFile(suffix=".pdf", delete=False) as temp_pdf:
        temp_pdf.write(response.content)
        temp_pdf_path = temp_pdf.name
    loader = PyPDFLoader(temp_pdf_path)
    documents = loader.load()
    os.remove(temp_pdf_path)
    doc_string = ""
    for doc in documents:
        doc_string += doc.page_content

    pdf_content = BytesIO(response.content)
    reader = PdfReader(pdf_content)
    metadata = reader.metadata
    article_title = metadata.title if metadata.title else ""
    return doc_string, article_title
pdf = load_pdf("https://arxiv.org/pdf/2307.06435")
len(pdf)

2

In [14]:
pdf[1]

'A Comprehensive Overview of Large Language Models'